In [8]:
import pandas as pd
from pathlib import Path

In [9]:
counts = pd.read_csv('1_clients/client_1/counts.tsv', sep='\t', index_col=0).T
design = pd.read_csv('1_clients/client_1/design.tsv', sep='\t', index_col=0)

In [10]:
data = pd.concat([design, counts], axis=1)
data

,Lum,Basal,is_LumA,diagnosis_age,stage,Entrezid:100130426,Entrezid:100133144,Entrezid:100134869,Entrezid:10357,Entrezid:10431,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR
TCGA-A2-A0CT-01,1,0,1,71.0,2,0,0,12,142,2155,...,254,5813,2505,63,2174,2539,3064,1936,152,0
TCGA-A2-A0D3-01,1,0,1,42.0,1,0,39,14,274,2688,...,317,3578,3745,268,2976,15067,5174,3718,315,0
TCGA-A2-A0ES-01,1,0,1,52.0,2,0,27,14,227,2151,...,231,1721,3558,149,2847,13444,3999,2752,486,0
TCGA-A2-A0YJ-01,0,1,0,39.0,3,0,14,19,380,2618,...,120,754,2906,263,2583,8318,4215,2282,219,0
TCGA-A7-A26I-01,0,1,0,65.0,2,0,54,21,334,5189,...,378,2797,3139,880,7162,20921,4847,4848,818,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-E9-A1NI-01,1,0,0,51.0,2,0,12,15,182,2696,...,101,580,1888,119,1314,4392,2003,1265,57,0
TCGA-E9-A22D-01,1,0,0,38.0,2,0,12,12,831,5316,...,236,2271,2007,403,2322,12012,2808,3087,1977,0
TCGA-E9-A295-01,1,0,1,71.0,2,1,1,8,146,2458,...,114,825,1475,139,1190,5040,1812,1060,1934,0
TCGA-EW-A1P4-01,0,1,0,43.0,2,0,3,7,345,3321,...,54,469,2825,246,1755,9149,2407,3832,33,0


In [11]:
def split_by_fractions(df: pd.DataFrame, fracs: list, random_state: int):
    df = df.sample(frac=1.0, random_state=random_state)
    assert sum(fracs) == 1.0, 'fractions sum is not 1.0 (fractions_sum={})'.format(sum(fracs))
    remain = df.index.copy().to_frame()
    res = []
    for i in range(len(fracs)):
        fractions_sum = sum(fracs[i:])
        frac = fracs[i] / fractions_sum
        idxs = remain.sample(frac=frac, random_state=random_state).index
        remain = remain.drop(idxs)
        res.append(idxs)
    return [df.loc[idxs] for idxs in res]

In [12]:
c, p1, p2, p3, p4 = split_by_fractions(data, fracs=[0.1, 0.15, 0.15, 0.3, 0.3], random_state=42)
display(c)
clients = [c, p1, p2, p3, p4]
for n_client in range(1, 6):
    Path(f'5_clients/client_{n_client}').mkdir(parents=True, exist_ok=True)
    counts = clients[n_client-1].drop(['Lum', 'Basal', 'is_LumA', 'diagnosis_age', 'stage'], axis=1).T
    design = clients[n_client-1].loc[:, ['Lum', 'Basal', 'is_LumA', 'diagnosis_age', 'stage']]
    counts.to_csv(f'5_clients/client_{n_client}/counts.tsv', index=True, sep='\t')
    design.to_csv(f'5_clients/client_{n_client}/design.tsv', index=True, sep='\t')


,Lum,Basal,is_LumA,diagnosis_age,stage,Entrezid:100130426,Entrezid:100133144,Entrezid:100134869,Entrezid:10357,Entrezid:10431,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR
TCGA-E2-A15F-01,1,0,1,64.0,1,0,41,12,213,2290,...,178,1811,4239,140,2529,5647,6733,2349,48,0
TCGA-AO-A1KR-01,0,1,0,51.0,2,0,0,11,196,3348,...,21,262,1561,811,1256,6407,1273,1086,44,0
TCGA-D8-A1JM-01,0,1,0,59.0,2,0,63,40,321,5814,...,174,1543,2897,1419,2445,12249,2665,4180,1150,3
TCGA-A7-A4SE-01,0,1,0,54.0,2,0,21,40,315,1176,...,111,758,2132,143,1790,16939,2559,1685,42,0
TCGA-E2-A15S-01,1,0,0,34.0,2,0,65,34,228,5266,...,376,3747,7197,732,4150,3040,5304,25280,972,0
TCGA-A2-A0ER-01,1,0,1,63.0,1,0,26,10,366,3073,...,272,2331,3567,133,3633,5167,6208,2836,56,0
TCGA-E2-A105-01,1,0,1,79.0,2,0,63,52,292,2942,...,238,2025,4205,248,3247,6649,2290,3422,112,0
TCGA-A7-A5ZW-01,1,0,1,47.0,2,0,0,11,409,2170,...,117,1177,1736,126,1961,9168,2184,2541,888,0
TCGA-BH-A0EA-01,1,0,1,72.0,2,0,10,21,164,1369,...,327,1497,2665,121,1800,15226,3746,2162,829,0
TCGA-AN-A0AS-01,1,0,1,70.0,3,0,8,21,218,1988,...,189,1287,3805,2,1586,8627,2451,1672,90,0
